## Tratamento de Dados

In [1]:
import numpy as np
import pandas as pd

**Fazendo a leitura de todos os Datasets**

In [2]:
bossa = pd.read_csv('dataset/bossa_nova.csv')
funk = pd.read_csv('dataset/funk.csv')
gospel = pd.read_csv('dataset/gospel.csv')
sertanejo = pd.read_csv('dataset/sertanejo.csv')

Para que o algoritmo possa identificar qual o tipo de letra, é preciso adicionar uma coluna 'label' que contenha para cada tipo de música um valor. Depois disso, pode-se unir os quatro conjuntos de dados em um só.

In [3]:
bossa['label'] = 0
funk['label'] = 1
gospel['label'] = 2
sertanejo['label'] = 3

In [4]:
datasets = [bossa,funk,gospel,sertanejo]

In [5]:
lyrics = pd.concat(datasets)

In [6]:
# É preciso usar reset_index para que os indexes sejam ajustados ao novo conjunto
lyrics.reset_index(drop=True,inplace=True)

In [7]:
lyrics.head()

,lyric,label
0,\nEu sei que vou te amar\nPor toda a minha vi...,0
1,\nOlha que coisa mais linda\nMais cheia de gr...,0
2,\nEra uma casa\nMuito engraçada\nNão tinha te...,0
3,"\nDe tudo, ao meu amor serei atento antes\nE ...",0
4,\nQuando a luz dos olhos meus\nE a luz dos ol...,0


Agora, é preciso fazer um processamento de texto nas letras das músicas para que somente palavras relevantes sejam usadas pelo algoritmo. Para isso, é preciso substituir as quebras de linha ('\n') por espaços, e remover símbolos de pontuação e stop words.

Convém então criar um método que execute todos esses passos e depois seja aplicado em todo o conjunto de dados.

**Obtendo o conjunto de stopwords, através da biblioteca nltk**

In [8]:
# É preciso baixar o conjunto de stopwords através do nltk.download_shell() -> d -> stopwords
from nltk.corpus import stopwords
stopwords.words('portuguese')

['de',
 'a',
 'o',
 'que',
 'e',
 'do',
 'da',
 'em',
 'um',
 'para',
 'com',
 'não',
 'uma',
 'os',
 'no',
 'se',
 'na',
 'por',
 'mais',
 'as',
 'dos',
 'como',
 'mas',
 'ao',
 'ele',
 'das',
 'à',
 'seu',
 'sua',
 'ou',
 'quando',
 'muito',
 'nos',
 'já',
 'eu',
 'também',
 'só',
 'pelo',
 'pela',
 'até',
 'isso',
 'ela',
 'entre',
 'depois',
 'sem',
 'mesmo',
 'aos',
 'seus',
 'quem',
 'nas',
 'me',
 'esse',
 'eles',
 'você',
 'essa',
 'num',
 'nem',
 'suas',
 'meu',
 'às',
 'minha',
 'numa',
 'pelos',
 'elas',
 'qual',
 'nós',
 'lhe',
 'deles',
 'essas',
 'esses',
 'pelas',
 'este',
 'dele',
 'tu',
 'te',
 'vocês',
 'vos',
 'lhes',
 'meus',
 'minhas',
 'teu',
 'tua',
 'teus',
 'tuas',
 'nosso',
 'nossa',
 'nossos',
 'nossas',
 'dela',
 'delas',
 'esta',
 'estes',
 'estas',
 'aquele',
 'aquela',
 'aqueles',
 'aquelas',
 'isto',
 'aquilo',
 'estou',
 'está',
 'estamos',
 'estão',
 'estive',
 'esteve',
 'estivemos',
 'estiveram',
 'estava',
 'estávamos',
 'estavam',
 'estivera',
 'es

**Obtendo conjunto de símbolos de pontuação, através da biblioteca de string do Python**

In [9]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

**Criando a função de processamento**

In [10]:
def processamento_texto(txt):
    # remover quebras de linha
    txt = txt.replace('\n',' ')
    # remover símbolos de pontuação, resultando em um array de caracteres
    txt = [char for char in txt if char not in string.punctuation]
    # depois, juntar os caracteres em palavras novamente e separá-los em uma lista de tokens
    txt = ''.join(txt).split()
    # por fim, remover as stopwords da lista
    txt = [word for word in txt if word.lower() not in stopwords.words('portuguese')]
    
    return txt

## Criação do Modelo

Para fazer a predição, é preciso seguir alguns passos:

1. Converter o texto das letras em uma lista de tokens únicos. Para isso, utiliza-se o Count Vectorizer do scikit, passando como analisador a função processamento_texto criada acima, para que as palavas sejam separadas de acordo com ela.
2. Obter os valores Tf-idf de todas as palavras. Este valor indica a importância de cada palavra em relação a todo o conjunto de palavras das letras de música. Para isso, utiliza-se o TfidfTransformer, que gerará os valores com base no que foi obtido no passo anterior.
3. Aplicar o modelo estatístico com base nos valores Tf-idf.

Para facilitar a execução destes passos, será utilizado a estrutura de Pipeline presente no scikit, que permite que uma série de transformações sejam passadas, de forma que elas são executadas em ordem.

**Separando o conjunto de dados em conjunto de treino e de teste**

In [11]:
from sklearn.model_selection import train_test_split
lyric_train,lyric_test,label_train,label_test = train_test_split(lyrics['lyric'],lyrics['label'],test_size=0.3)

**Utilizando Naive-Bayes**

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

pipeline = Pipeline([
    ('bow',CountVectorizer(analyzer=processamento_texto)),
    ('tfidf',TfidfTransformer()),
    ('nb',MultinomialNB())
])

In [13]:
pipeline.fit(lyric_train,label_train)

Pipeline(memory=None,
     steps=[('bow', CountVectorizer(analyzer=<function processamento_texto at 0x7fe839f7f730>,
        binary=False, decode_error='strict', dtype=<class 'numpy.int64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=1.0,
        max_features=None, min_df=1, ngram_range=(1, 1), preproce...linear_tf=False, use_idf=True)), ('nb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [14]:
predictions = pipeline.predict(lyric_test)

In [15]:
from sklearn.metrics import classification_report

In [16]:
print(classification_report(label_test,predictions))

             precision    recall  f1-score   support

          0       0.93      0.61      0.73       219
          1       0.95      0.75      0.84       253
          2       0.85      0.94      0.90       236
          3       0.63      0.90      0.74       252

avg / total       0.84      0.80      0.80       960



**Utilizando Support Vector Machines**

In [44]:
from sklearn.svm import SVC

pipelineSVC = Pipeline([
    ('bow',CountVectorizer(analyzer=processamento_texto)),
    ('tfidf',TfidfTransformer()),
    ('svc',SVC())
])

In [45]:
pipelineSVC.fit(lyric_train,label_train)

Pipeline(memory=None,
     steps=[('bow', CountVectorizer(analyzer=<function processamento_texto at 0x7fe839f7f730>,
        binary=False, decode_error='strict', dtype=<class 'numpy.int64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=1.0,
        max_features=None, min_df=1, ngram_range=(1, 1), preproce...,
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

In [46]:
predictions = pipelineSVC.predict(lyric_test)

In [47]:
print(classification_report(label_test,predictions))

             precision    recall  f1-score   support

          0       0.23      1.00      0.37       219
          1       0.00      0.00      0.00       253
          2       0.00      0.00      0.00       236
          3       0.00      0.00      0.00       252

avg / total       0.05      0.23      0.08       960



/home/luis/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [51]:
from sklearn.model_selection import GridSearchCV

In [56]:
param_grid = {'svc__C':[0.1,1,10,1000],'svc__gamma':[1,0.1,0.01,0.001,0.0001]}

In [57]:
grid = GridSearchCV(pipelineSVC,param_grid,verbose=3)

In [58]:
grid.fit(lyric_train,label_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] svc__C=0.1, svc__gamma=1 ........................................
[CV]  svc__C=0.1, svc__gamma=1, score=0.25935828877005346, total= 1.0min
[CV] svc__C=0.1, svc__gamma=1 ........................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.8min remaining:    0.0s


[CV]  svc__C=0.1, svc__gamma=1, score=0.26104417670682734, total= 1.1min
[CV] svc__C=0.1, svc__gamma=1 ........................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  3.6min remaining:    0.0s


[CV]  svc__C=0.1, svc__gamma=1, score=0.25906040268456376, total= 1.0min
[CV] svc__C=0.1, svc__gamma=0.1 ......................................
[CV]  svc__C=0.1, svc__gamma=0.1, score=0.25935828877005346, total= 1.1min
[CV] svc__C=0.1, svc__gamma=0.1 ......................................
[CV]  svc__C=0.1, svc__gamma=0.1, score=0.2597054886211513, total= 1.1min
[CV] svc__C=0.1, svc__gamma=0.1 ......................................
[CV]  svc__C=0.1, svc__gamma=0.1, score=0.25906040268456376, total= 1.0min
[CV] svc__C=0.1, svc__gamma=0.01 .....................................
[CV]  svc__C=0.1, svc__gamma=0.01, score=0.25935828877005346, total= 1.0min
[CV] svc__C=0.1, svc__gamma=0.01 .....................................
[CV]  svc__C=0.1, svc__gamma=0.01, score=0.2597054886211513, total= 1.0min
[CV] svc__C=0.1, svc__gamma=0.01 .....................................
[CV]  svc__C=0.1, svc__gamma=0.01, score=0.25906040268456376, total= 1.0min
[CV] svc__C=0.1, svc__gamma=0.001 ................

[CV]  svc__C=1000, svc__gamma=0.0001, score=0.789261744966443, total= 1.2min


[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed: 104.4min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('bow', CountVectorizer(analyzer=<function processamento_texto at 0x7fe839f7f730>,
        binary=False, decode_error='strict', dtype=<class 'numpy.int64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=1.0,
        max_features=None, min_df=1, ngram_range=(1, 1), preproce...,
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'svc__C': [0.1, 1, 10, 1000], 'svc__gamma': [1, 0.1, 0.01, 0.001, 0.0001]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [59]:
grid.best_params_

{'svc__C': 1000, 'svc__gamma': 1}

In [60]:
grid_predictions = grid.predict(lyric_test)

In [61]:
print(classification_report(label_test,grid_predictions))

             precision    recall  f1-score   support

          0       0.83      0.83      0.83       219
          1       0.86      0.82      0.84       253
          2       0.90      0.90      0.90       236
          3       0.77      0.81      0.79       252

avg / total       0.84      0.84      0.84       960



In [81]:
from sklearn.externals import joblib

In [82]:
joblib.dump(grid, "svc.pkl")

['svc.pkl']